# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")

Mounted at /content/gdrive


In [ ]:
%pip install causal_conv1d

In [ ]:
os.environ['MAX_JOBS'] = '3'
%pip install ninja
%pip install --no-build-isolation --verbose ./selective_scan

In [3]:
import torch, InitMamba
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

In [26]:
import importlib
importlib.reload(InitMamba)

reload
reload


<module 'InitMamba' from "/content/gdrive/Othercomputers/retep's PC/MambaVAE/InitMamba/__init__.py">

In [49]:
import VAE
importlib.reload(VAE)
from VAE import MambaVAE

# Lab

In [56]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
tokenizer.add_special_tokens({'bos_token': '<|startoftext|>'})
inputs = tokenizer(["<|startoftext|>Today is Monday, Tomorrow is Tuesday.<|endoftext|>",
           "<|startoftext|>Today is Sunday, Tomorrow is Monday.<|endoftext|>",
           "<|startoftext|>Today is Sunday, Tomorrow is Monday.<|endoftext|>"],
          #   max_length = 32,
          #  padding = 'max_length',
           return_tensors='pt').to('cuda')
empty_inputs = tokenizer(["<|startoftext|>", "<|startoftext|>", "<|startoftext|>"],
          #   max_length = 32,
          #  padding = 'max_length',
           return_tensors='pt').to('cuda')

In [114]:
inputs2 = tokenizer(["<|startoftext|>The maximum amount of time you allow the computation to run for in seconds. generation will still finish the current pass after allocated time has been passed.<|endoftext|>",
           "<|startoftext|>The maximum amount of time you allow the computation to run for in seconds. generation will still finish the current pass after allocated time has been passed.<|endoftext|>",
           "<|startoftext|>The maximum amount of time you allow the computation to run for in seconds. generation will still finish the current pass after allocated time has been passed.<|endoftext|>"], return_tensors='pt').to('cuda')

In [50]:
vae = MambaVAE().cuda()
vae.load_state_dict(torch.load('results/result6/model.pth', weights_only=True))

<All keys matched successfully>

In [124]:
vae.eval()
res = vae(**inputs2)
print(tokenizer.batch_decode(res[2].argmax(-1)))
state2 = res[3]

['The maximum amount of time you allow the computation to run for in generation. generation will still finish the current pass after allocated time has been passed.<|endoftext|><|endoftext|>', 'The maximum amount of time you allow the computation to run for in generation. generation will still finish the current pass after allocated time has been passed.<|endoftext|><|endoftext|>', 'The maximum amount of time you allow the computation to run for in generation. generation will still finish the current pass after allocated time has been passed.<|endoftext|><|endoftext|>']


In [127]:
out = vae.decoder.generate(**empty_inputs, inputs_ssm_states = state2,
              tokenizer = tokenizer, stop_strings = '<|endoftext|>',
              bad_words_ids = [[187]], max_length = 100, num_beams=5)
tokenizer.batch_decode(out, skip_special_tokens=True)

['The maximum amount of time you allow the computation to run for in generation will generation always the current pass after allocation time has been passed.',
 'The maximum amount of time you allow the computation to run for in generation will generation always the current pass after allocation time has been passed.',
 'The maximum amount of time you allow the computation to run for in generation will generation always the current pass after allocation time has been passed.']

In [ ]:
model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
model1.eval()

res1 = model1(**inputs, output_ssm_last_states = True)
ssm_last_states = res1.ssm_last_states
print(ssm_last_states[-1])

In [ ]:
out = model1.generate(**empty_inputs, inputs_ssm_states = ssm_last_states)
print(tokenizer.batch_decode(out))

In [ ]:
res = model1(**inputs, output_ssm_last_states = True, inputs_ssm_states = ssm_last_states)
print(res.logits)
tokenizer.batch_decode(res.logits.argmax(-1))

tensor([[[  9.9198,   1.5491,   2.6419,  ...,   1.6821,   1.6572,   1.5468],
         [  4.7165, -12.5163,   3.3999,  ..., -12.7500, -12.6262, -12.6619],
         [-19.7026, -46.9598, -22.1777,  ..., -46.7554, -46.8826, -46.8495],
         ...,
         [ -9.2194,   7.6863, -16.2589,  ...,   7.3706,   7.2829,   7.6166],
         [-13.6607, -18.8684, -18.2374,  ..., -18.5683, -18.7283, -18.9281],
         [  3.5942, -16.5568,   2.2905,  ..., -16.8151, -17.3365, -16.8327]],

        [[ 19.3583,  -0.6694,  13.4369,  ...,  -0.8893,  -0.7612,  -0.9876],
         [-11.5463, -25.2096, -10.4256,  ..., -25.2801, -25.2087, -25.3942],
         [ -0.7773, -17.1977,  -6.9741,  ..., -17.1582, -17.1418, -17.0683],
         ...,
         [ -2.7754,  -6.5402, -10.4917,  ...,  -6.7587,  -7.0094,  -6.6512],
         [ 10.1860, -15.5430,  -6.6677,  ..., -15.2001, -15.6132, -15.4658],
         [  7.5376, -31.5117,   2.6149,  ..., -31.8905, -32.1699, -31.5726]],

        [[ 19.3583,  -0.6694,  13.4369,  ...

['ethod and firstloatenselyariju our d',
 'namese and theirante Sbben T d',
 'namese and theirante Sbben T d']

In [ ]:
torch.autograd.grad(outputs=res.logits.mean(), inputs=ssm_last_states)

(tensor([[[ 0.2946,  1.0394,  0.3819,  ...,  1.8289, -0.8378,  0.7556],
          [-3.9220,  0.2882,  0.6721,  ...,  0.0723,  2.0089, -0.6532],
          [-0.4168, -0.7216,  0.0279,  ..., -0.8134,  0.4345,  0.0042],
          ...,
          [-0.0890,  0.5959, -0.7744,  ..., -1.4096, -0.1136, -1.0288],
          [ 0.5794,  0.3463,  0.4186,  ..., -0.6908,  0.4392,  0.0123],
          [-0.6525, -1.0777,  3.2442,  ...,  0.8828,  2.3689,  0.8545]],
 
         [[-0.0501,  1.0625,  0.1401,  ...,  0.2090,  0.0199, -0.0977],
          [-1.2707,  0.3201,  0.0247,  ...,  0.0203,  0.7554, -0.2175],
          [-0.0379, -0.1715,  0.0076,  ..., -0.1561, -0.4653,  0.0226],
          ...,
          [ 0.3033,  0.0045, -0.0945,  ...,  0.0733, -0.3215,  0.1298],
          [-0.0877, -0.0674, -0.0232,  ..., -0.0895,  0.1850,  0.2670],
          [-0.2333, -0.1446,  0.8714,  ...,  0.3004,  0.5954,  0.1813]],
 
         [[-0.0501,  1.0625,  0.1401,  ...,  0.2090,  0.0199, -0.0977],
          [-1.2707,  0.3201,

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
import torch
(torch.randn(768, 32)*0.5).pow(2).sum()

tensor(253.1772)